In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [2]:
# 데이터로드 (ch4-2(붓꽃데이터).csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('datasets/국민건강정보/NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [3]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [5]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [6]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [7]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx

[13,
 56,
 85,
 106,
 172,
 176,
 181,
 194,
 204,
 213,
 216,
 243,
 280,
 283,
 288,
 348,
 392,
 451,
 462,
 463,
 464,
 483,
 485,
 528,
 555,
 556,
 581,
 593,
 611,
 662,
 678,
 693,
 731,
 747,
 769,
 800,
 802,
 816,
 847,
 875,
 890,
 949,
 975,
 1012,
 1051,
 1060,
 1063,
 1064,
 1067,
 1087,
 1089,
 1090,
 1095,
 1102,
 1114,
 1199,
 1209,
 1210,
 1223,
 1244,
 1252,
 1254,
 1285,
 1294,
 1305,
 1337,
 1355,
 1366,
 1375,
 1392,
 1446,
 1448,
 1472,
 1487,
 1503,
 1514,
 1527,
 1532,
 1552,
 1571,
 1592,
 1620,
 1642,
 1657,
 1672,
 1688,
 1703,
 1713,
 1768,
 1828,
 1895,
 1912,
 1925,
 1942,
 1953,
 1985,
 2016,
 2035,
 2045,
 2055,
 2057,
 2074,
 2079,
 2089,
 2113,
 2154,
 2169,
 2211,
 2281,
 2288,
 2300,
 2312,
 2315,
 2334,
 2336,
 2338,
 2399,
 2403,
 2425,
 2431,
 2453,
 2469,
 2483,
 2484,
 2515,
 2524,
 2537,
 2550,
 2558,
 2559,
 2567,
 2611,
 2625,
 2673,
 2700,
 2719,
 2776,
 2784,
 2795,
 2839,
 2887,
 2892,
 2901,
 2916,
 2920,
 2921,
 2953,
 2962,
 2980,
 29

In [8]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,2,13,150.0,55.0,86.0,0.4,0.4,1.0,1.0,138.0,...,85.0,59.0,167.0,1.0,0.6,28.0,23.0,21.0,1.0,0.0
1,2,6,155.0,65.0,74.0,0.3,0.3,1.0,1.0,99.0,...,56.0,61.0,57.0,1.0,0.6,19.0,5.0,10.0,1.0,1.0
2,1,12,180.0,70.0,83.0,1.0,0.5,1.0,1.0,119.0,...,108.0,60.0,122.0,1.0,1.0,17.0,11.0,25.0,3.0,1.0
3,2,11,150.0,60.0,87.5,1.5,1.5,1.0,1.0,132.0,...,389.0,43.0,89.0,1.0,0.7,25.0,29.0,18.0,1.0,0.0
4,1,7,170.0,65.0,76.1,0.5,0.7,1.0,1.0,102.0,...,50.0,54.0,83.0,1.0,1.0,25.0,20.0,17.0,1.0,1.0
5,1,8,180.0,100.0,96.0,1.0,1.0,1.0,1.0,108.0,...,268.0,71.0,118.0,1.0,1.1,29.0,25.0,82.0,3.0,1.0
6,1,9,160.0,55.0,69.0,1.2,1.0,1.0,1.0,100.0,...,125.0,45.0,133.0,1.0,1.0,18.0,16.0,26.0,3.0,0.0
7,1,14,165.0,70.0,89.0,0.8,0.6,1.0,1.0,127.0,...,180.0,41.0,162.0,1.0,0.9,18.0,23.0,20.0,2.0,1.0
8,2,10,165.0,75.0,82.0,1.5,1.2,1.0,1.0,120.0,...,167.0,34.0,92.0,1.0,0.8,18.0,15.0,20.0,1.0,0.0
9,1,9,180.0,85.0,91.0,0.6,1.2,1.0,1.0,135.0,...,183.0,56.0,173.0,1.0,0.8,24.0,33.0,39.0,3.0,1.0


In [9]:
rawData_sample.shape

(50000, 22)

In [10]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [11]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [12]:
# RMSLE 계산하는 사용자정의 함수
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values) :
    # 넘파이로 배열 형태로 변환
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌움
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 함
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균값 구함
    mean_difference = difference.mean()
    
    # 다시 루트를 씌움
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [13]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
import xgboost as xgb # xgboost 기법에 관련된 모듈

gbm = xgb.XGBClassifier(max_depth=4, n_estimators=300, learning_rate=0.05)

gbm

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=4,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)

In [14]:
# KFold 교차검증
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 트레이닝셋에 대한 KFold 교차검증 수
%time score = cross_val_score(gbm, X_train, y_train, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

# 0에 근접할수록 좋은 데이터
print("Score= {0:.5f}".format(score))

C:\Users\a\anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:42:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:42:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:42:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:42:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [15]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
gbm.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = gbm.predict(X_test)

[19:43:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(gbm.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(gbm.score(X_test, y_test)))

Train Set Score1 : 0.76
Test Set Score1 : 0.74


In [17]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.74


In [18]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : gbm.feature_importances_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=False)

,중요도
흡연상태,0.437937
성별코드,0.231210
연령대코드(5세단위),0.078713
신장(5Cm단위),0.075388
감마지티피,0.041702
HDL콜레스테롤,0.025126
(혈청지오티)ALT,0.013724
LDL콜레스테롤,0.008824
(혈청지오티)AST,0.008501
체중(5Kg 단위),0.008174
